In [ ]:
import ollama
import random
import time
import json
import os

In [ ]:
# Hilfsfunktionen / IO

def load_list_from_file(filename):
    with open(filename, 'r', encoding='utf-8') as file:
        return [line.strip() for line in file if line.strip()]

def generate_date(raw=False):
    tag = random.choice(datum_list)
    monat = random.choice(monat_list)
    jahr = random.choice(jahr_list)
    return (tag, monat, jahr) if raw else f"im {tag} {monat} {jahr}"

def find_span(text, target):
    start = text.find(target)
    return (start, start + len(target)) if start != -1 else None

In [ ]:
def build_prompt_type1(name, ort, verb_sample, datum_str):
    return (
        f"Erstelle einen deutschen Satz über die Person '{name}', "
        f"die sich in '{ort}' aufhält oder dort etwas tut. "
        f"Verwende exakt diesen Namen: {name}, und exakt diesen Ort: {ort}. "
        f"Nutze genau eines der folgenden Verben: {', '.join(verb_sample)}. "
        f"Der Satz muss das Datum {datum_str} enthalten, allerdings nicht am Satzanfang – "
        f"es soll möglichst in der Satzmitte oder am Ende erscheinen. "
        f"Gib ausschließlich den vollständigen Satz aus – keine Erklärungen, Kommentare oder Einleitungen."
    )

In [ ]:

def build_prompt_type2(name, ort):
    return (
        f"Schreibe einen deutschen Satz aus der Ich-Perspektive. "
        f"In diesem Satz beschreibe ich, wie ich zusammen mit '{name}' etwas in '{ort}' tue. "
        f"Der zeitliche Kontext liegt zwischen 1819 und 1894. "
        f"Verwende exakt diesen Namen: {name}, und exakt diesen Ort: {ort}. "
        f"Gib ausschließlich den vollständigen Satz aus – keine Erklärungen, Kommentare oder Einleitungen."
    )

In [ ]:
# Daten laden

person_list_1 = load_list_from_file('input/person_list.txt')
person_list_2 = load_list_from_file('input/nachname_list.txt')  # neu eingebunden
ortsname_list_1 = load_list_from_file('input/ortsname_list.txt')
ortsname_list_2 = load_list_from_file('input/ortsname_SA_E_list.txt')  # neu eingebunden
jahr_list = load_list_from_file("input/jahr_list.txt")
monat_list = load_list_from_file("input/monat_list.txt")
datum_list = load_list_from_file("input/datum_list.txt")
verb_list = load_list_from_file("input/verb_list.txt")

In [ ]:
# Ausgabedatei
os.makedirs("output", exist_ok=True)
RAW_JSONL_PATH = "training_data_raw.jsonl"

In [ ]:
# Generator-Funktionen

def generate_examples_type1(model_name="llama3", max_retries=6, verbs_per_prompt=10):
    random.shuffle(person_list_1)
    random.shuffle(ortsname_list_1)
    ort_cycle = (ortsname_list_1 * ((len(person_list_1) // len(ortsname_list_1)) + 1))[:len(person_list_1)]

    count_ok = 0
    for name, ort in zip(person_list_1, ort_cycle):
        attempts = 0
        while attempts < max_retries:
            tag, monat, jahr = generate_date(raw=True)
            datum_str = f"im {tag} {monat} {jahr}"
            verb_sample = random.sample(verb_list, min(verbs_per_prompt, len(verb_list)))
            prompt = build_prompt_type1(name, ort, verb_sample, datum_str)

            try:
                result = ollama.generate(model=model_name, prompt=prompt)
                satz = result['response'].strip()

                if name in satz and ort in satz and jahr in satz:
                    entities = []

                    span = find_span(satz, name)
                    if span: entities.append([span[0], span[1], "PERSON"])

                    span = find_span(satz, ort)
                    if span: entities.append([span[0], span[1], "LOC"])

                    full = f"{tag} {monat} {jahr}"
                    span = find_span(satz, full) or find_span(satz, jahr)
                    if span: entities.append([span[0], span[1], "DATE"])

                    if entities:
                        yield satz, entities
                        print(f"[Typ1] {satz}")
                        count_ok += 1
                        break
                attempts += 1
            except Exception as e:
                print(f"Fehler: {e}")
                time.sleep(1)
                attempts += 1
    print(f"Typ1: {count_ok} Beispiele erfolgreich erzeugt.")

def generate_examples_type2(model_name="llama3", max_retries=6):
    random.shuffle(person_list_2)
    random.shuffle(ortsname_list_2)
    ort_cycle = (ortsname_list_2 * ((len(person_list_2) // len(ortsname_list_2)) + 1))[:len(person_list_2)]

    count_ok = 0
    for name, ort in zip(person_list_2, ort_cycle):
        attempts = 0
        while attempts < max_retries:
            prompt = build_prompt_type2(name, ort)
            try:
                result = ollama.generate(model=model_name, prompt=prompt)
                satz = result['response'].strip()

                if name in satz and ort in satz:
                    entities = []

                    span = find_span(satz, name)
                    if span: entities.append([span[0], span[1], "PERSON"])

                    span = find_span(satz, ort)
                    if span: entities.append([span[0], span[1], "LOC"])

                    # Zeitspanne 1819-1894 → prüfe, ob eine Jahreszahl drin ist
                    for year in map(str, range(1819, 1895)):
                        span = find_span(satz, year)
                        if span:
                            entities.append([span[0], span[1], "DATE"])
                            break

                    if entities:
                        yield satz, entities
                        print(f"[Typ2] {satz}")
                        count_ok += 1
                        break
                attempts += 1
            except Exception as e:
                print(f"Fehler: {e}")
                time.sleep(1)
                attempts += 1
    print(f"Typ2: {count_ok} Beispiele erfolgreich erzeugt.")



In [ ]:
# Hauptfunktion
def generate_all():
    with open(RAW_JSONL_PATH, "w", encoding="utf-8") as f:
        for satz, entities in generate_examples_type1():
            f.write(json.dumps([satz, entities], ensure_ascii=False) + "\n")
        for satz, entities in generate_examples_type2():
            f.write(json.dumps([satz, entities], ensure_ascii=False) + "\n")

    print(f"Alle Daten gespeichert in: {RAW_JSONL_PATH}")

if __name__ == "__main__":
    generate_all()
